In [1]:
import streamlit as st
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline

2022-09-25 16:30:43.171 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [2]:
df = pd.read_csv("without_outliers_autoscout.csv")   

In [3]:
features = ["make_model", "body_type", "km", "age", "gearing_type", "gears","price_€"]
df = df[features]

In [14]:
df[features]

,make_model,body_type,km,age,gearing_type,gears,price_€
0,Audi A1,Sedans,56013.000000,3.0,Automatic,7.0,15770.0
1,Audi A1,Sedans,80000.000000,2.0,Automatic,7.0,14500.0
2,Audi A1,Sedans,83450.000000,3.0,Automatic,7.0,14640.0
3,Audi A1,Sedans,73000.000000,3.0,Automatic,6.0,14500.0
4,Audi A1,Sedans,16200.000000,3.0,Automatic,6.0,16790.0
...,...,...,...,...,...,...,...
15913,Renault Espace,Van,16.987484,0.0,Automatic,6.0,39950.0
15914,Renault Espace,Van,9900.000000,0.0,Automatic,7.0,39885.0
15915,Renault Espace,Van,15.000000,0.0,Automatic,6.0,39875.0
15916,Renault Espace,Van,10.000000,0.0,Automatic,6.0,39700.0


In [4]:
X = df.drop(columns = ["price_€"])
y = df["price_€"]


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [6]:
cat = X.select_dtypes("object").columns
cat = list(cat)

In [7]:
column_trans = make_column_transformer((OneHotEncoder(handle_unknown="ignore", sparse=False), cat), 
                                       remainder=MinMaxScaler())


In [8]:
X_train=column_trans.fit_transform(X_train)

X_test=column_trans.transform(X_test)

In [9]:
pipe_model = Pipeline([("OneHotEncoder", column_trans), ("Lasso", Lasso(alpha = 0.01))])
pipe_model.fit(X, y)


C:\Users\hbpbn\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.566e+08, tolerance: 8.671e+07
  model = cd_fast.enet_coordinate_descent(


Pipeline(steps=[('OneHotEncoder',
                 ColumnTransformer(remainder=MinMaxScaler(),
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['make_model', 'body_type',
                                                   'gearing_type'])])),
                ('Lasso', Lasso(alpha=0.01))])

In [10]:
import pickle
pickle.dump(pipe_model, open('autoscout_deployment_project', 'wb'))



In [11]:
model = pickle.load(open('autoscout_deployment_project', 'rb'))


In [12]:
df

,make_model,body_type,km,age,gearing_type,gears,price_€
0,Audi A1,Sedans,56013.000000,3.0,Automatic,7.0,15770.0
1,Audi A1,Sedans,80000.000000,2.0,Automatic,7.0,14500.0
2,Audi A1,Sedans,83450.000000,3.0,Automatic,7.0,14640.0
3,Audi A1,Sedans,73000.000000,3.0,Automatic,6.0,14500.0
4,Audi A1,Sedans,16200.000000,3.0,Automatic,6.0,16790.0
...,...,...,...,...,...,...,...
15913,Renault Espace,Van,16.987484,0.0,Automatic,6.0,39950.0
15914,Renault Espace,Van,9900.000000,0.0,Automatic,7.0,39885.0
15915,Renault Espace,Van,15.000000,0.0,Automatic,6.0,39875.0
15916,Renault Espace,Van,10.000000,0.0,Automatic,6.0,39700.0


In [13]:
df.describe()

,km,age,gears,price_€
count,15918.000000,15918.000000,15918.000000,15918.000000
mean,31965.211041,1.392260,5.940256,18022.598882
std,37071.059519,1.120012,0.728105,7380.981051
min,0.000000,0.000000,5.000000,4950.000000
25%,2000.000000,0.000000,5.000000,12850.000000
50%,20408.500000,1.000000,6.000000,16900.000000
75%,46899.250000,2.000000,6.000000,21900.000000
max,317000.000000,3.000000,9.000000,74600.000000


In [21]:
dicti = {"make_model":["Audi A3"],
         "body_type" : ["Sedan"],
         "km" : [200000],
         "age" : [2],
         "gearing_type" : ["Automatic"],
         "gears": [6]}

In [22]:
df2 = pd.DataFrame(dicti)
df2

,make_model,body_type,km,age,gearing_type,gears
0,Audi A3,Sedan,200000,2,Automatic,6


In [24]:
model.predict(df2)

array([15551.13659821])